In [1]:
import torch
import torch.nn as nn
from torch.autograd import  Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_dataset = dsets.MNIST(root='./data', train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data', train=False,
                           transform=transforms.ToTensor(),
                           download=True)

In [19]:
train_dataset.train_data.size()

torch.Size([60000, 28, 28])

In [3]:
batch_size = 100
n_iters = 3000

n_epochs = n_iters/(len(train_dataset)/ batch_size)
n_epochs = int(n_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [20]:
class CNNMode(nn.Module):
    def __init__(self):
        super(CNNMode, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
    
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32*7*7, 10)
    
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        return out
    

In [21]:
model = CNNMode()

In [22]:
criterion = nn.CrossEntropyLoss()

In [23]:
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [24]:
print(model.parameters())

print(len(list(model.parameters())))
#convolution 1 parameters
print(list(model.parameters())[0].size())

#Convolution 1 bias
print(list(model.parameters())[1].size())

#convolution  2 parameters
print(list(model.parameters())[2].size())

#convolution  2 bias
print(list(model.parameters())[3].size())

#FC parameters
print(list(model.parameters())[4].size())

#FC bias
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x7fbb2f564bf8>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


In [27]:
iter = 0
for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #load images
        
        images = Variable(images)
        labels = Variable(labels)
        
        #clear gradients
        optimizer.zero_grad()
        
        #forward pass to get output/logits
        
        outputs = model(images)
        
        #calculate loss
        
        loss = criterion(outputs, labels)
        
        #getting gradients wrt parameters
        loss.backward()
        
        #update parameters
        
        optimizer.step()
        
        iter +=1
        if iter% 500 ==0:
            #calculate accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
            accuracy = 100* correct/total
            print('iteration {}, Loss {}, accuraracy {}'.format(iter, loss.data, accuracy))

iteration 500, Loss 0.032077837735414505, accuraracy 98
iteration 1000, Loss 0.06039522960782051, accuraracy 98
iteration 1500, Loss 0.04308474436402321, accuraracy 98
iteration 2000, Loss 0.011060094460844994, accuraracy 98
iteration 2500, Loss 0.050186775624752045, accuraracy 97
iteration 3000, Loss 0.02036641538143158, accuraracy 98
